In [129]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from lxml import etree
import requests
import csv
import pandas as pd

In [130]:

def check_exists_by_xpath(xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except:
        return False
    return True


def scroll_to_bottom(iteration=None):
    xpath_sidebar = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
    if(iteration == None):
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", xpath_sidebar)
        time.sleep(2)
    else:
        for i in range(1, iteration):
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", xpath_sidebar)
            time.sleep(2)

def url_hit(data_latitude, data_longitude) :
    try : 
        data = []
        for latitude, longitude in zip (data_latitude, data_longitude) :
            start_time = time.time()
            try : 
                url = f'https://www.google.com/maps/search/Bulutangkis/@{latitude},{longitude},12.58z?entry=ttu' 
                print(f"Start in {url}")
                driver.get(url)
                
                time.sleep(2)
                
                try :
                    for i in range(1, 100):
                        # scroll down to bottom sidebar
                        scroll_to_bottom()
                        
                        # xpath_endsidebar = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[243]/div/p/span/span')
                        if(check_exists_by_xpath('/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[243]/div/p/span/span')):
                            scroll_to_bottom(2)
                            print("End scroll down !!")
                            try :
                                # Dapatkan HTML halaman web setelah di-render
                                html = driver.page_source
                                # Gunakan BeautifulSoup untuk mengekstrak data
                                soup = BeautifulSoup(html, 'lxml')
                                tes = soup.find_all('div', class_='bJzME tTVLSc')
                                #start crawling data !
                                for cari in tes:
                                    hrefs = cari.find_all('a', class_='hfpxzc')
                                    for href in hrefs:
                                        href_value = href.get('href')
                                        data.append({'URL': href_value} if href_value else {'URL': '-'})
                                        
                                    poi_names = cari.find_all('div', class_='qBF1Pd fontHeadlineSmall')
                                    for poi in poi_names:
                                        poinames_value = poi.text.strip() 
                                        data.append({'Name': poinames_value} if poinames_value else {'Name': '-'})

                                    ratings = cari.find_all('span', class_='MW4etd')
                                    for rating in ratings:
                                        ratings_value = rating.text.strip()
                                        data.append({'Rating': ratings_value} if ratings_value else {'Rating': '-'})
                                    
                                    address_schema = cari.find_all('div', class_='Nv2PK tH5CWc THOPZb')
                                    found_addresses = set()
                                    for address_testing in address_schema :                
                                        tagsaddress_elements = cari.find_all('div', class_='W4Efsd')
                                        for tagsaddress1 in tagsaddress_elements :
                                            tagsaddress1_value = tagsaddress1.find_all('div', class_='W4Efsd')
                                            for tagsaddress2 in tagsaddress1_value :
                                                tagsaddress2_value = tagsaddress2.findParents('div', class_='W4Efsd')
                                                for tagsaddress3 in tagsaddress2_value :
                                                    tagsaddress3_value = tagsaddress3.text.strip()
                                                    # Kondisi untuk memeriksa apakah alamat sudah ditemukan sebelumnya
                                                    if tagsaddress3_value not in found_addresses:
                                                        found_addresses.add(tagsaddress3_value)
                                                        data.append({'Address': tagsaddress3_value} if tagsaddress3_value else {'Address': '-'})

                            except Exception as e :
                                print("Terdapat error dibagian Crawling data(Function scrape_data)")
                            break
                
                except Exception as e :
                    print(f"Ada error ni di request {url} -> {e}")
                    
                print(f'url {url} sudah selesai load')              
    
            except TimeoutError :
                print(f'Teeett habis waktu jadi error di {url}')
                
            df = pd.DataFrame(data, columns=['URL','Name', 'Rating', 'Address'])
            df.head()
            print(data)
            df.to_csv(r'C:\Users\bputra\OneDrive - ESRI Indonesia\PT ESRI Indonesia\Research\code-script\17_scrapping_gmaps\gmaps\output\testing1_scrape_gmaps_bulutangkis',index=False, encoding='utf-8-sig',sep=';')
                
            end_time = time.time()
            
            execution_time = (end_time - start_time)/60
            print(f"Waktu yang dibutuhkan {execution_time:.2f} menit")
        
        print("Data sudah selesai di scrapping, horee !!!")
    
    except Exception as e :
        print('error nih haduhhh')


In [131]:
#data = r"D:\SCRIPT - ARCPY - PYTHON\script-python-working-in-esri\17_scrapping_gmaps\gmaps\input\kabkot_list_and_xy_sample.csv"
data = r"C:\Users\bputra\OneDrive - ESRI Indonesia\PT ESRI Indonesia\Research\code-script\17_scrapping_gmaps\gmaps\input\kabkot_list_and_xy_bulutangkis.csv"
df = pd.read_csv(data, delimiter=";")
pd.DataFrame(df)

,kakot,Shape_Leng,Shape_Area,lat,long
0,Kota Bogor,0.708177,0.009106,-6.593494,106.799357
1,Kota Padang,1.881366,0.056664,-0.899532,100.437912
2,Magelang,2.016873,0.093674,-7.502515,110.249411
3,Pacitan,2.715767,0.117653,-8.124997,111.178150


In [132]:
data_lat = df['lat']
data_long = df['long']

In [133]:
driver = webdriver.Chrome()
url_hit(data_lat, data_long)
time.sleep(2)
driver.quit()

Start in https://www.google.com/maps/search/Bulutangkis/@-6.59349435,106.7993574,12.58z?entry=ttu
End scroll down !!
url https://www.google.com/maps/search/Bulutangkis/@-6.59349435,106.7993574,12.58z?entry=ttu sudah selesai load
[{'URL': 'https://www.google.com/maps/place/Lapangan+Bulutangkis+Cilibende+RT.02%2FRW.06/data=!4m7!3m6!1s0x2e69c42ccc465753:0x28f30e572581bc14!8m2!3d-6.5865014!4d106.8099665!16s%2Fg%2F11bwp7319z!19sChIJU1dGzCzEaS4RFLyBJVcO8yg?authuser=0&hl=id&rclk=1'}, {'URL': 'https://www.google.com/maps/place/Lapangan+Badminton+Bougenville/data=!4m7!3m6!1s0x2e69c4fcedf0a5d5:0x7513151341011bd8!8m2!3d-6.5848358!4d106.7610346!16s%2Fg%2F11hck13vpv!19sChIJ1aXw7fzEaS4R2BsBQRMVE3U?authuser=0&hl=id&rclk=1'}, {'URL': 'https://www.google.com/maps/place/Lapangan+PB+BIMA+KUD+%28Bulutangkis%29/data=!4m7!3m6!1s0x2e69c350a36d31e3:0x4556fcb46b3d09ed!8m2!3d-6.5452168!4d106.7286674!16s%2Fg%2F11mvxr8mgc!19sChIJ4zFto1DDaS4R7Qk9a7T8VkU?authuser=0&hl=id&rclk=1'}, {'URL': 'https://www.google.com/ma